***
# `COVID-19 Dashboard`
***
<!-- ***
# <font color='blue'>COVID-19 Dashboard</font>
*** -->

> <font color='blue'>Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus. Most people infected with 
the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older 
people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and 
cancer are more likely to develop serious illness. The best way to prevent and slow down transmission is be well informed about 
the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands 
or using an alcohol based rub frequently and not touching your face.</font> 

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
import plotly.graph_objects as go

In [2]:
HTML("<img src=https://www.ikg-wien.at/wp-content/uploads/2020/03/Coronavirus-2.jpg, width=650 >")

In [3]:
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [4]:
#death_df.head(2)

In [5]:
#confirmed_df.head(2)

In [6]:
#recovered_df.head(2)

In [7]:
#country_df.head(2)

In [8]:
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)

In [9]:
confirmed_df = confirmed_df.rename(columns = {'province/state':'state', 'country/region':'country'})
recovered_df = recovered_df.rename(columns = {'province/state':'state', 'country/region':'country'})
death_df = death_df.rename(columns = {'province/state':'state', 'country/region':'country'})
country_df = country_df.rename(columns = {'country_region':'country'})

In [10]:
sorted_country_df = country_df.sort_values('confirmed', ascending=False)

In [11]:
#sorted_country_df.head(5)

***
# `Corona virus spread over Map`
***

In [12]:
world_map = folium.Map(location=[15,2], tiles='openstreetmap', zoom_start=2, max_zoom= 6, min_zoom=2)

for i in range(len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        fill_color = 'blue',
        color = 'red',
        tooltip = "<div style='margin: 0; background-color: powderblue; color: blue;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: blue;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

    
world_map    

***
# `Current cases`
***

# Select the country to get data:

In [13]:
def country_data(country):
    df_list = [confirmed_df, death_df, recovered_df]
    l=[]
    for df in df_list:       
        if country == 'World' or country == 'world':
                data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
                l.append(data[-1])                
        else:
            data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            l.append(data[-1])
    display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(l[0]) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(l[1]) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(l[2]) + "</span>"+
             "</div>")
               )
cou=list(confirmed_df.country)
cou.insert(0,'World')
cou=list(dict.fromkeys(cou))
interact(country_data, country=cou);


interactive(children=(Dropdown(description='country', options=('World', 'Afghanistan', 'Albania', 'Algeria', '…

In [14]:
# def highlight_col(x):
#     r = 'background-color: red'
#     p = 'background-color: purple'
#     g = 'background-color: green'
#     temp_df = pd.DataFrame('', index=x.index, columns = x.columns)
#     temp_df.iloc[:,4]=p
#     temp_df.iloc[:,5]=r
#     temp_df.iloc[:,6]=g
#     return temp_df
# sorted_country_df.head(5).style.apply(highlight_col, axis=None)


# Slide to compare data of countries:

In [15]:
data_frame=sorted_country_df[['country','confirmed','deaths','recovered']]
data_frame.index=[x+1 for x in range(len(data_frame.values))]
def highlight_col(x):
    y = 'background-color: yellow'
    r = 'background-color: red'
    g = 'background-color: lightgreen'
    temp_df = pd.DataFrame('', index=x.index, columns = x.columns)
    temp_df.iloc[:,1]=y
    temp_df.iloc[:,2]=r
    temp_df.iloc[:,3]=g
    return temp_df
def get_highlight_col(N):
    return data_frame.head(N).style.apply(highlight_col, axis=None)
interact(get_highlight_col, N=(1,30));

interactive(children=(IntSlider(value=15, description='N', max=30, min=1), Output()), _dom_classes=('widget-in…

In [16]:
import plotly.express as px

# Slide to compare worst hit countries:

In [17]:
def worst_hit(Number):
    fig = px.scatter(sorted_country_df.head(Number), x='country', y='confirmed', size='confirmed', color='country', 
                     hover_name='country', size_max=60)
    fig.update_layout(
        title=" Worst hit countries",
        xaxis_title="Countries",
        yaxis_title="Confirmed Cases",
        width = 800
    )
    fig.show()
#worst_hit(10)
interact(worst_hit, Number=(1,30));


interactive(children=(IntSlider(value=15, description='Number', max=30, min=1), Output()), _dom_classes=('widg…

# Select the country to get data:

In [18]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
  
    fig.show();
#plot_cases_of_a_country('India') 
cou=list(confirmed_df.country)
cou.insert(0,'World')
cou=list(dict.fromkeys(cou))
interact(plot_cases_of_a_country, country=cou);
    
    

interactive(children=(Dropdown(description='country', options=('World', 'Afghanistan', 'Albania', 'Algeria', '…

***
# <font color='yellow'>Top 10 worst affected countries- Confirmed cases</font>
***

In [19]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    #title= "Top 10 worst affected countries- Confirmed cases", # the axis names
    color_discrete_sequence=["yellow"], 
    height=500,
    width=800
)

***
# <font color='red'>Top 10 worst affected countries- Death/Confirmed ratio</font>
***

In [20]:
country_df['d/c']=country_df['deaths']/country_df['confirmed']
d= country_df.sort_values('d/c', ascending=False)
#sorted_death_df = country_df.sort_values(country_df['d/c'], ascending=False)
px.bar(
    d.head(10),
    x = "country",
    y = "deaths",
    #title= "Top 10 worst affected countries- Death cases", # the axis names
    color_discrete_sequence=["red"], 
    height=500,
    width=800
)


In [21]:
#d=np.array(country_df['deaths']/country_df['confirmed'])
#d.sort_values(ascending=True)
#country_df['d/c']=country_df['deaths']/country_df['confirmed']
#type(country_df['d/c'])
#type(country_df['deaths'])
#country_df.columns
#d= country_df.sort_values('d/c', ascending=True)

***
# <font color='green'>Top 10 worst affected countries-Recovered/Confirmed ratio</font>
***

In [22]:
country_df['r/c']=country_df['recovered']/country_df['confirmed']
r= country_df.sort_values('r/c', ascending=False)
#sorted_recovered_df = country_df.sort_values('recovered', ascending=True)
px.bar(
    r.head(10),
    x = "country",
    y = "recovered",
    #title= "Top 10 worst affected countries-Recovered cases", # the axis names
    color_discrete_sequence=["green"], 
    height=500,
    width=800
)

In [23]:
#![Markdown Logo is here.](https://www.bioworld.com/ext/resources/Stock-images/Therapeutic-topics/Infectious/Coronavirus-
#pandemic2.png?1588893047)

In [24]:
# color="255,157,180"  # coral

# def print_color(c):
#     source = "<h1 style='color: rgb({0})'>Color Text</h1>".format(c)
#     return HTML(source)

# print_color(color)

In [25]:
# color="255,157,180" 
# HTML("<h1 style='color: rgb({0})'>COVID-19</h1>".format(color))

In [26]:
#<font color='red'>bar</font>

In [27]:
# likes=0
# button = widgets.Button(description="Like")
# output = widgets.Output()
# text=widgets.Text()
# display(button, output)
# display(text)

# def on_button_clicked(b):
#     global likes
#     likes=likes+1
#     text.value='Thank you....\nLikes = '+str(likes)

# button.on_click(on_button_clicked)
# #print(likes)

### If you liked my work, please press the like button below:

In [28]:
# file=open("likes.txt","r+")
# likes=file.readline()
# file.close
# button = widgets.Button(description="Like")
# output = widgets.Output()
# text=widgets.Text()
# display(button, output)
# display(text)

# def on_button_clicked(b):
#     global likes
#     like=int(likes)+1
#     text.value='Thank you....\nLikes = '+str(like)
#     likes=str(like)
#     file=open("likes.txt","w+")
#     file.write(likes)

# button.on_click(on_button_clicked)

In [29]:
# import urllib.request
# site='https://raw.githubusercontent.com/kara006n/c-19_dashboard/master/likes.txt'
# u=urllib.request.urlopen(site)
# text=u.read()
# p=str(text)
# print(p)